# Stacking

Несмотря на то, что в открытом доступе существует довольно много реализаций стекинга, некоторые из которых даже представлены в виде библиотечных функций, лучше сделать собственную. Стекинг - не классический алгоритм решения задачи, а скорее набор правил для помощи в решении задачи другим алгоритмам. Если вы серьезно займетесь машинным обучением, рано или поздно вам скорее всего захочется что-нибудь поменять в этом наборе правил, поэтому собственная реализация с понятным вам кодом будет как нельзя кстати.

В этом домашнем задании вы создадите собственную функцию для блендинга/классического стекинга моделей, которая пригодится вам в дальнейшем.

Ниже вы увидите заготовку для функции, где:

- models - список объектов базовых алгоритмов

- meta_alg - мета-алгоритм

data_train, targets_train, data_test, targets_test - тренировочные и тестовые признаки и целевые переменные

- test_size - размер тестовых данных для блендинга в промежутке (0, 1)

- cv - количество разбиений для кросс-валидации

In [5]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [38]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

# def stacking(models, meta_alg, data_train, targets_train, data_test, targets_test=None, test_size=None, cv=5):
#     
#     x_train, valid, y_train, valid_true = train_test_split(data_train,
#                                                       targets_train,
#                                                       train_size=0.6,
#                                                       random_state=17)
#     
#     meta_mtrx = np.empty((y_train.shape[0], len(models)))
#     folds = StratifiedKFold(n_splits=cv)
#     for n, model in enumerate(models):
#         # meta_mtrx[:,n] = model.fit(data_train, targets_train).predict(data_test)
#         meta_mtrx[:,n] = cross_val_predict(model, x_train, y_train, cv=folds)
#         
#           
#     meta_model = meta_alg.fit(meta_mtrx, y_train)
#     meta_mtrx_test = np.empty((data_test.shape[0], len(models)))
#     
#     for n, model in enumerate(models):
#         meta_mtrx_test[:, n] = model.fit(data_train, targets_train).predict(data_test)
# 
#     meta = meta_model.predict(meta_mtrx_test)
#     print(roc_auc_score(targets_test,meta))
#     return meta
#     

# titanic = pd.read_csv('9.7_titanic.csv')
# targets = titanic.Survived
# data = titanic.drop(columns='Survived')
# 
# x_train, x_test, y_train, y_test = train_test_split(data,
#                                                     targets,
#                                                     train_size=0.8,
#                                                     random_state=17)
# models = [LogisticRegression(random_state=0, max_iter=10000),
#           GradientBoostingClassifier(random_state=0),
#           KNeighborsClassifier(n_neighbors=3)]
# 
# estimator = RandomForestClassifier(random_state=0)
# x = stacking(models, estimator, x_train, y_train, x_test, y_test)


0.7943913155854225


### 1.
В следующей ячейке в теле функции определен условный оператор if-else. После elif вместо pass пропишите код из лекции с некоторыми новыми вставками в таком порядке: деление data_train и targets_train на тренировочные и валидационные данные с помощью функции train_test_split, где test_size=test_size из определения функции; определение матрицы meta_mtrx; цикл, в котором заполняется meta_mtrx: сначала строка, где модель обучается с помощью метода fit на тренировочных данных из разбиения, затем строка, где матрица заполняется значениями предсказаний моделей на валидационных данных. На этом цикл заканчивается.

После цикла добавьте строку с методом fit мета-алгоритма: обучите его на значениях полученной матрицы meta_mtrx и целевой переменной для валидационных данных.

Определите матрицу meta_mtrx_test. Напишите цикл, где эта матрица заполняется предсказаниями базовых моделей на тестовых данных data_test.

После цикла сделайте предсказания мета-алгоритма для значений матрицы meta_mtrx_test.

Дополните код еще одним оператором if, который будет проверять, существуют ли данные targets_test для проверки качества работы модели на тестовых данных: если targets_test не является None, тогда выведите метрику roc_auc_score для предсказаний мета-алгоритма на тестовых данных.

In [39]:
def stacking(models, meta_alg, data_train, targets_train, data_test, targets_test=None, random_state=None, test_size=None, cv=5):
    if test_size is None:
        x_train, valid, y_train, valid_true = train_test_split(data_train,
                                                      targets_train,
                                                      train_size=0.6,
                                                      random_state=random_state or 0)
    
        meta_mtrx = np.empty((y_train.shape[0], len(models)))
        folds = StratifiedKFold(n_splits=cv)
        for n, model in enumerate(models):
            meta_mtrx[:,n] = cross_val_predict(model, x_train, y_train, cv=folds)
            
              
        meta_model = meta_alg.fit(meta_mtrx, y_train)
        meta_mtrx_test = np.empty((data_test.shape[0], len(models)))
        
        for n, model in enumerate(models):
            meta_mtrx_test[:, n] = model.fit(data_train, targets_train).predict(data_test)
    
        meta = meta_model.predict(meta_mtrx_test)
        if targets_test is not None:
            print(roc_auc_score(targets_test,meta))
            
        return meta

    elif test_size > 0 and test_size < 1:
        x_train, valid, y_train, valid_true = train_test_split(data_train,
                                                      targets_train,
                                                      test_size=test_size,
                                                      random_state=random_state or 0)
    
        meta_mtrx = np.empty((y_train.shape[0], len(models)))
        folds = StratifiedKFold(n_splits=cv)
        for n, model in enumerate(models):
            meta_mtrx[:,n] = cross_val_predict(model, x_train, y_train, cv=folds)
            
              
        meta_model = meta_alg.fit(meta_mtrx, y_train)
        meta_mtrx_test = np.empty((data_test.shape[0], len(models)))
        
        for n, model in enumerate(models):
            meta_mtrx_test[:, n] = model.fit(data_train, targets_train).predict(data_test)
    
        meta = meta_model.predict(meta_mtrx_test)
        if targets_test is not None:
            print(roc_auc_score(targets_test,meta))
            
        return meta

    else:
        raise ValueError("test_size must be between 0 and 1")

### 2.
Теперь напишите код стекинга вместо pass после оператора if.

Сразу определите матрицу meta_mtrx. Напишите цикл для заполнения матрицы: сначала напишите строку, где каждый столбец метапризнаков (для каждой модели) заполняется с помощью функции cross_val_predict(base_algotithm, data_train, targets_train, cv, method='predict'); следующая строка - обучите каждый базовый алгоритм на полном тренировочном датасете.

После цикла обучите мета-алгоритм на матрице метапризнаков meta_mtrx. Определите матрицу meta_mtrx_test.

Напишите второй цикл, где матрица meta_mtrx_test заполняется предсказаниями базовых моделей на тестовых данных data_test.

После цикла сделайте предсказания мета-алгоритма для значений матрицы meta_mtrx_test.

Так же, как и для блендинга, напишите код проверки для targets_test и выведите roc_auc_score, если это возможно.

In [ ]:
# ваш код

### 3.
Базовая функция стекинга готова. Теперь проверим, как она работает. Ниже представлен уже знакомый нам датасет Titanic, разделенный на тренировочный и тестовый датасеты; предопределенные базовые алгоритмы и мета-алгоритм. Ваша задача - составить список базовых алгоритмов и запустить функцию в трех различных вариантах (при этом в каждом из них все значения data_train, targets_train, data_test, targets_test должны быть определены):

1. Вызвать исключение "test_size must be between 0 and 1".

2. Установить test_size=0.3; вывести AUC и массив полученных предсказаний.

3. Оставить test_size=None; вывести AUC и массив полученных предсказаний.

In [40]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier


titanic = pd.read_csv('9.7_titanic.csv')
targets = titanic.Survived
data = titanic.drop(columns='Survived')

x_train, x_test, y_train, y_test = train_test_split(data,
                                                    targets,
                                                    train_size=0.8,
                                                    random_state=17)

knn = KNeighborsClassifier(n_neighbors=3)
lr = LogisticRegression(random_state=17)
svc = SVC(random_state=17)

meta = XGBClassifier(n_estimators=40)

my_stack = stacking([knn,lr,svc], meta, x_train, y_train, x_test, y_test, 42, 0.3)

0.7434091496510726


По мере того, как вы будете использовать эту функцию, вам могут пригодиться такие дополнительные параметры как: random_state, который позволит делать воспроизводимые модели; metrics - список метрик, которые могут быть вычислены; grid_search, который может производить поиск лучших параметров для алгоритмов, и т.д.